In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

Path to dataset files: /Users/anand/.cache/kagglehub/datasets/dylanjcastillo/7k-books-with-metadata/versions/3


In [6]:
# Here we are importing pandas to read the csv file 
import pandas as pd

book_df = pd.read_csv(f"{path}/books.csv")
book_df.head(1)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0


In [7]:
# Getting the overviw of the data 
book_df.shape

(6810, 12)

In [9]:
# Checking if the dataset has any missing value 
book_df.isnull().sum()

isbn13               0
isbn10               0
title                0
subtitle          4429
authors             72
categories          99
thumbnail          329
description        262
published_year       6
average_rating      43
num_pages           43
ratings_count       43
dtype: int64